# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ KISS-GP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SKI and stochastic variational regression to rapidly train using minibatches on the `song` UCI dataset, which has hundreds of thousands of training examples.

Stochastic variational inference has several major advantages over the standard regression setting. Most notably, the ELBO used for optimization decomposes in such a way that stochastic gradient descent techniques can be used. See https://arxiv.org/pdf/1411.2005.pdf and https://arxiv.org/pdf/1611.00336.pdf for more technical details of this.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `3droad` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=2028, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. To use grid interpolation for variational inference, we'll be using a `GridInterpolationVariationalStrategy`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

See the CIFAR example for using an `AbstractVariationalGP` with an `AdditiveGridInterpolationVariationalStrategy`, which additionally assumes the kernel decomposes additively, which is a strong modelling assumption but allows us to use many more output features from the neural network.

In [5]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution, GridInterpolationVariationalStrategy

class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, grid_size=32, grid_bounds=[(-1, 1), (-1, 1)]):
        variational_distribution = CholeskyVariationalDistribution(num_inducing_points=grid_size*grid_size)
        variational_strategy = GridInterpolationVariationalStrategy(self,
                                                                    grid_size=grid_size,
                                                                    grid_bounds=grid_bounds,
                                                                    variational_distribution=variational_distribution)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [6]:
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer()
        self.grid_bounds = grid_bounds
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features)
        return res

model = DKLModel(feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalELBO`), call backwards, and do a step of optimization.

In [7]:
model.train()
likelihood.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 7

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

for i in range(num_epochs):
    scheduler.step()
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):        
        optimizer.zero_grad()
        
        # Because the grid is relatively small, we turn off the Toeplitz matrix multiplication and just perform them directly
        # We find this to be more efficient when the grid is very small.
        with gpytorch.settings.use_toeplitz(False):
            output = model(x_batch)
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

        # The actual optimization step
        loss.backward()
        optimizer.step()

Epoch 1 [0/172] - Loss: 256.829 [-256.829, 0.000, 0.000]
Epoch 1 [1/172] - Loss: 237.597 [-237.228, 0.369, 0.000]
Epoch 1 [2/172] - Loss: 226.816 [-226.364, 0.452, 0.000]
Epoch 1 [3/172] - Loss: 201.177 [-200.882, 0.295, 0.000]
Epoch 1 [4/172] - Loss: 184.058 [-183.559, 0.499, 0.000]
Epoch 1 [5/172] - Loss: 182.979 [-182.470, 0.509, 0.000]
Epoch 1 [6/172] - Loss: 187.616 [-187.123, 0.493, 0.000]
Epoch 1 [7/172] - Loss: 169.386 [-168.979, 0.407, 0.000]
Epoch 1 [8/172] - Loss: 145.485 [-145.099, 0.386, 0.000]
Epoch 1 [9/172] - Loss: 144.235 [-143.850, 0.384, 0.000]
Epoch 1 [10/172] - Loss: 139.877 [-139.527, 0.349, 0.000]
Epoch 1 [11/172] - Loss: 133.140 [-132.759, 0.381, 0.000]
Epoch 1 [12/172] - Loss: 120.182 [-119.813, 0.369, 0.000]
Epoch 1 [13/172] - Loss: 117.984 [-117.599, 0.385, 0.000]
Epoch 1 [14/172] - Loss: 111.975 [-111.598, 0.377, 0.000]
Epoch 1 [15/172] - Loss: 102.585 [-102.230, 0.355, 0.000]
Epoch 1 [16/172] - Loss: 111.100 [-110.779, 0.322, 0.000]
Epoch 1 [17/172] - Loss:

Epoch 1 [148/172] - Loss: 30.508 [-30.501, 0.008, 0.000]
Epoch 1 [149/172] - Loss: 29.303 [-29.296, 0.008, 0.000]
Epoch 1 [150/172] - Loss: 29.396 [-29.388, 0.008, 0.000]
Epoch 1 [151/172] - Loss: 30.438 [-30.431, 0.008, 0.000]
Epoch 1 [152/172] - Loss: 27.519 [-27.511, 0.008, 0.000]
Epoch 1 [153/172] - Loss: 27.981 [-27.974, 0.008, 0.000]
Epoch 1 [154/172] - Loss: 28.627 [-28.619, 0.008, 0.000]
Epoch 1 [155/172] - Loss: 29.085 [-29.077, 0.008, 0.000]
Epoch 1 [156/172] - Loss: 27.740 [-27.733, 0.008, 0.000]
Epoch 1 [157/172] - Loss: 26.016 [-26.008, 0.008, 0.000]
Epoch 1 [158/172] - Loss: 27.154 [-27.146, 0.008, 0.000]
Epoch 1 [159/172] - Loss: 28.167 [-28.159, 0.008, 0.000]
Epoch 1 [160/172] - Loss: 29.723 [-29.715, 0.008, 0.000]
Epoch 1 [161/172] - Loss: 30.179 [-30.171, 0.008, 0.000]
Epoch 1 [162/172] - Loss: 28.494 [-28.486, 0.008, 0.000]
Epoch 1 [163/172] - Loss: 28.793 [-28.786, 0.008, 0.000]
Epoch 1 [164/172] - Loss: 29.919 [-29.912, 0.007, 0.000]
Epoch 1 [165/172] - Loss: 29.67

Epoch 2 [123/172] - Loss: 24.248 [-24.242, 0.006, 0.000]
Epoch 2 [124/172] - Loss: 20.942 [-20.936, 0.006, 0.000]
Epoch 2 [125/172] - Loss: 24.795 [-24.789, 0.006, 0.000]
Epoch 2 [126/172] - Loss: 19.729 [-19.723, 0.006, 0.000]
Epoch 2 [127/172] - Loss: 23.454 [-23.448, 0.006, 0.000]
Epoch 2 [128/172] - Loss: 21.089 [-21.084, 0.006, 0.000]
Epoch 2 [129/172] - Loss: 21.803 [-21.797, 0.006, 0.000]
Epoch 2 [130/172] - Loss: 21.630 [-21.624, 0.006, 0.000]
Epoch 2 [131/172] - Loss: 21.022 [-21.016, 0.006, 0.000]
Epoch 2 [132/172] - Loss: 19.342 [-19.336, 0.006, 0.000]
Epoch 2 [133/172] - Loss: 19.359 [-19.353, 0.006, 0.000]
Epoch 2 [134/172] - Loss: 20.994 [-20.988, 0.006, 0.000]
Epoch 2 [135/172] - Loss: 20.218 [-20.213, 0.006, 0.000]
Epoch 2 [136/172] - Loss: 19.204 [-19.198, 0.006, 0.000]
Epoch 2 [137/172] - Loss: 19.247 [-19.242, 0.006, 0.000]
Epoch 2 [138/172] - Loss: 20.682 [-20.676, 0.006, 0.000]
Epoch 2 [139/172] - Loss: 18.650 [-18.644, 0.006, 0.000]
Epoch 2 [140/172] - Loss: 20.23

Epoch 3 [98/172] - Loss: 17.575 [-17.570, 0.005, 0.000]
Epoch 3 [99/172] - Loss: 17.622 [-17.617, 0.005, 0.000]
Epoch 3 [100/172] - Loss: 18.544 [-18.539, 0.005, 0.000]
Epoch 3 [101/172] - Loss: 18.757 [-18.751, 0.005, 0.000]
Epoch 3 [102/172] - Loss: 17.213 [-17.208, 0.005, 0.000]
Epoch 3 [103/172] - Loss: 17.495 [-17.490, 0.005, 0.000]
Epoch 3 [104/172] - Loss: 17.658 [-17.653, 0.005, 0.000]
Epoch 3 [105/172] - Loss: 17.331 [-17.325, 0.005, 0.000]
Epoch 3 [106/172] - Loss: 17.369 [-17.364, 0.005, 0.000]
Epoch 3 [107/172] - Loss: 16.952 [-16.947, 0.005, 0.000]
Epoch 3 [108/172] - Loss: 17.033 [-17.027, 0.005, 0.000]
Epoch 3 [109/172] - Loss: 17.710 [-17.704, 0.005, 0.000]
Epoch 3 [110/172] - Loss: 16.958 [-16.953, 0.005, 0.000]
Epoch 3 [111/172] - Loss: 16.872 [-16.866, 0.005, 0.000]
Epoch 3 [112/172] - Loss: 17.300 [-17.295, 0.005, 0.000]
Epoch 3 [113/172] - Loss: 17.915 [-17.910, 0.005, 0.000]
Epoch 3 [114/172] - Loss: 17.622 [-17.617, 0.005, 0.000]
Epoch 3 [115/172] - Loss: 17.768 

Epoch 4 [72/172] - Loss: 14.878 [-14.873, 0.005, 0.000]
Epoch 4 [73/172] - Loss: 16.017 [-16.012, 0.005, 0.000]
Epoch 4 [74/172] - Loss: 14.951 [-14.946, 0.005, 0.000]
Epoch 4 [75/172] - Loss: 15.057 [-15.052, 0.005, 0.000]
Epoch 4 [76/172] - Loss: 15.156 [-15.151, 0.005, 0.000]
Epoch 4 [77/172] - Loss: 15.764 [-15.759, 0.005, 0.000]
Epoch 4 [78/172] - Loss: 14.770 [-14.765, 0.005, 0.000]
Epoch 4 [79/172] - Loss: 14.672 [-14.667, 0.005, 0.000]
Epoch 4 [80/172] - Loss: 14.766 [-14.761, 0.005, 0.000]
Epoch 4 [81/172] - Loss: 14.994 [-14.989, 0.005, 0.000]
Epoch 4 [82/172] - Loss: 14.294 [-14.289, 0.005, 0.000]
Epoch 4 [83/172] - Loss: 15.211 [-15.206, 0.005, 0.000]
Epoch 4 [84/172] - Loss: 14.892 [-14.887, 0.005, 0.000]
Epoch 4 [85/172] - Loss: 15.003 [-14.998, 0.005, 0.000]
Epoch 4 [86/172] - Loss: 14.874 [-14.869, 0.005, 0.000]
Epoch 4 [87/172] - Loss: 15.171 [-15.166, 0.005, 0.000]
Epoch 4 [88/172] - Loss: 16.078 [-16.073, 0.005, 0.000]
Epoch 4 [89/172] - Loss: 15.450 [-15.445, 0.005,

Epoch 5 [47/172] - Loss: 14.843 [-14.838, 0.005, 0.000]
Epoch 5 [48/172] - Loss: 14.207 [-14.202, 0.005, 0.000]
Epoch 5 [49/172] - Loss: 15.258 [-15.253, 0.005, 0.000]
Epoch 5 [50/172] - Loss: 14.153 [-14.148, 0.005, 0.000]
Epoch 5 [51/172] - Loss: 15.142 [-15.137, 0.005, 0.000]
Epoch 5 [52/172] - Loss: 14.871 [-14.866, 0.005, 0.000]
Epoch 5 [53/172] - Loss: 15.065 [-15.060, 0.005, 0.000]
Epoch 5 [54/172] - Loss: 15.160 [-15.155, 0.005, 0.000]
Epoch 5 [55/172] - Loss: 14.170 [-14.165, 0.005, 0.000]
Epoch 5 [56/172] - Loss: 14.545 [-14.540, 0.005, 0.000]
Epoch 5 [57/172] - Loss: 16.148 [-16.143, 0.005, 0.000]
Epoch 5 [58/172] - Loss: 15.529 [-15.524, 0.005, 0.000]
Epoch 5 [59/172] - Loss: 16.636 [-16.631, 0.005, 0.000]
Epoch 5 [60/172] - Loss: 15.653 [-15.648, 0.005, 0.000]
Epoch 5 [61/172] - Loss: 15.160 [-15.155, 0.005, 0.000]
Epoch 5 [62/172] - Loss: 15.225 [-15.220, 0.005, 0.000]
Epoch 5 [63/172] - Loss: 16.252 [-16.247, 0.005, 0.000]
Epoch 5 [64/172] - Loss: 14.766 [-14.761, 0.005,

Epoch 6 [21/172] - Loss: 14.758 [-14.753, 0.005, 0.000]
Epoch 6 [22/172] - Loss: 15.541 [-15.536, 0.005, 0.000]
Epoch 6 [23/172] - Loss: 15.430 [-15.425, 0.005, 0.000]
Epoch 6 [24/172] - Loss: 15.219 [-15.214, 0.005, 0.000]
Epoch 6 [25/172] - Loss: 14.020 [-14.015, 0.005, 0.000]
Epoch 6 [26/172] - Loss: 15.153 [-15.148, 0.005, 0.000]
Epoch 6 [27/172] - Loss: 14.537 [-14.532, 0.005, 0.000]
Epoch 6 [28/172] - Loss: 14.698 [-14.693, 0.005, 0.000]
Epoch 6 [29/172] - Loss: 14.308 [-14.303, 0.005, 0.000]
Epoch 6 [30/172] - Loss: 14.975 [-14.970, 0.005, 0.000]
Epoch 6 [31/172] - Loss: 14.486 [-14.481, 0.005, 0.000]
Epoch 6 [32/172] - Loss: 14.628 [-14.623, 0.005, 0.000]
Epoch 6 [33/172] - Loss: 14.744 [-14.739, 0.005, 0.000]
Epoch 6 [34/172] - Loss: 14.793 [-14.788, 0.005, 0.000]
Epoch 6 [35/172] - Loss: 15.050 [-15.045, 0.005, 0.000]
Epoch 6 [36/172] - Loss: 14.431 [-14.426, 0.005, 0.000]
Epoch 6 [37/172] - Loss: 14.232 [-14.227, 0.005, 0.000]
Epoch 6 [38/172] - Loss: 15.110 [-15.105, 0.005,

Epoch 6 [167/172] - Loss: 14.602 [-14.597, 0.005, 0.000]
Epoch 6 [168/172] - Loss: 15.289 [-15.284, 0.005, 0.000]
Epoch 6 [169/172] - Loss: 15.052 [-15.047, 0.005, 0.000]
Epoch 6 [170/172] - Loss: 14.700 [-14.695, 0.005, 0.000]
Epoch 6 [171/172] - Loss: 14.735 [-14.730, 0.005, 0.000]
Epoch 7 [0/172] - Loss: 16.148 [-16.143, 0.005, 0.000]
Epoch 7 [1/172] - Loss: 15.029 [-15.024, 0.005, 0.000]
Epoch 7 [2/172] - Loss: 15.121 [-15.116, 0.005, 0.000]
Epoch 7 [3/172] - Loss: 14.881 [-14.876, 0.005, 0.000]
Epoch 7 [4/172] - Loss: 15.741 [-15.736, 0.005, 0.000]
Epoch 7 [5/172] - Loss: 14.679 [-14.674, 0.005, 0.000]
Epoch 7 [6/172] - Loss: 14.327 [-14.322, 0.005, 0.000]
Epoch 7 [7/172] - Loss: 14.861 [-14.856, 0.005, 0.000]
Epoch 7 [8/172] - Loss: 14.087 [-14.082, 0.005, 0.000]
Epoch 7 [9/172] - Loss: 14.838 [-14.833, 0.005, 0.000]
Epoch 7 [10/172] - Loss: 15.112 [-15.107, 0.005, 0.000]
Epoch 7 [11/172] - Loss: 14.695 [-14.690, 0.005, 0.000]
Epoch 7 [12/172] - Loss: 15.467 [-15.463, 0.005, 0.00

Epoch 7 [141/172] - Loss: 15.246 [-15.241, 0.005, 0.000]
Epoch 7 [142/172] - Loss: 15.023 [-15.018, 0.005, 0.000]
Epoch 7 [143/172] - Loss: 14.906 [-14.901, 0.005, 0.000]
Epoch 7 [144/172] - Loss: 14.576 [-14.571, 0.005, 0.000]
Epoch 7 [145/172] - Loss: 14.708 [-14.703, 0.005, 0.000]
Epoch 7 [146/172] - Loss: 14.376 [-14.371, 0.005, 0.000]
Epoch 7 [147/172] - Loss: 15.070 [-15.065, 0.005, 0.000]
Epoch 7 [148/172] - Loss: 14.989 [-14.984, 0.005, 0.000]
Epoch 7 [149/172] - Loss: 14.475 [-14.470, 0.005, 0.000]
Epoch 7 [150/172] - Loss: 15.362 [-15.357, 0.005, 0.000]
Epoch 7 [151/172] - Loss: 13.407 [-13.402, 0.005, 0.000]
Epoch 7 [152/172] - Loss: 13.776 [-13.771, 0.005, 0.000]
Epoch 7 [153/172] - Loss: 14.092 [-14.087, 0.005, 0.000]
Epoch 7 [154/172] - Loss: 14.999 [-14.994, 0.005, 0.000]
Epoch 7 [155/172] - Loss: 14.373 [-14.368, 0.005, 0.000]
Epoch 7 [156/172] - Loss: 14.579 [-14.574, 0.005, 0.000]
Epoch 7 [157/172] - Loss: 15.382 [-15.377, 0.005, 0.000]
Epoch 7 [158/172] - Loss: 14.71

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [8]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

In [9]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 9.143630027770996
